In [1]:
pip install cmake

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install dlib

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install imageio

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install h5py

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install imutils

Note: you may need to restart the kernel to use updated packages.


In [31]:
pip install opencv-python==3.4.0.14

  Using cached opencv-python-3.4.0.14.tar.gz (87.3 MB)
  ERROR: Command errored out with exit status 1:
   command: /usr/local/anaconda3/bin/python -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/7d/14zq0r5s0ybgxzpmh5vv_hz00000gn/T/pip-install-juxg4h_7/opencv-python_fa7244d881674ac0b4f7785a64d561fc/setup.py'"'"'; __file__='"'"'/private/var/folders/7d/14zq0r5s0ybgxzpmh5vv_hz00000gn/T/pip-install-juxg4h_7/opencv-python_fa7244d881674ac0b4f7785a64d561fc/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/7d/14zq0r5s0ybgxzpmh5vv_hz00000gn/T/pip-wheel-_76ir9on
       cwd: /private/var/folders/7d/14zq0r5s0ybgxzpmh5vv_hz00000gn/T/pip-install-juxg4h_7/opencv-python_fa7244d881674ac0b4f7785a64d561fc/
  C

Failed to build opencv-python
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.5.2.54
    Uninstalling opencv-python-4.5.2.54:
      Successfully uninstalled opencv-python-4.5.2.54
    Running setup.py install for opencv-python ... error
    ERROR: Command errored out with exit status 1:
     command: /usr/local/anaconda3/bin/python -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/7d/14zq0r5s0ybgxzpmh5vv_hz00000gn/T/pip-install-juxg4h_7/opencv-python_fa7244d881674ac0b4f7785a64d561fc/setup.py'"'"'; __file__='"'"'/private/var/folders/7d/14zq0r5s0ybgxzpmh5vv_hz00000gn/T/pip-install-juxg4h_7/opencv-python_fa7244d881674ac0b4f7785a64d561fc/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install mlxtend

Note: you may need to restart the kernel to use updated packages.


In [10]:
from alertness_imports import *
from playsound import playsound
import IPython

In [11]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter serverextension enable voila --sys-prefix

In [27]:
def eye_aspect_ratio(eye):
    """Calculates EAR -> ratio length and width of eyes"""
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def mouth_aspect_ratio(mouth):
    """Calculates MAR -> ratio length and width of mouth"""
    A = distance.euclidean(mouth[14], mouth[18])
    C = distance.euclidean(mouth[12], mouth[16])
    mar = (A ) / (C)
    return mar

def pupil_circularity(eye):
    """Calculates PUC -> low perimeter leads to lower pupil"""
    A = distance.euclidean(eye[1], eye[4])
    radius  = A/2.0
    Area = math.pi * (radius ** 2)
    p = 0
    p += distance.euclidean(eye[0], eye[1])
    p += distance.euclidean(eye[1], eye[2])
    p += distance.euclidean(eye[2], eye[3])
    p += distance.euclidean(eye[3], eye[4])
    p += distance.euclidean(eye[4], eye[5])
    p += distance.euclidean(eye[5], eye[0])
    return 4 * math.pi * Area / (p**2)

def mouth_over_eye(eye):
    """Calculates the MOE -> ratio of MAR to EAR"""
    ear = eye_aspect_ratio(eye)
    mar = mouth_aspect_ratio(eye)
    mouth_eye = mar/ear
    return mouth_eye

def sound_alarm(path):
    """play an alarm sound"""
    playsound.playsound(path)
    
def overlay_transparent(background_img, img_to_overlay_t, x, y, overlay_size=None):
    bg_img = background_img.copy()
    if overlay_size is not None:
        img_to_overlay_t = cv2.resize(img_to_overlay_t.copy(), overlay_size)
        
    # Extract the alpha mask of the RGBA image, convert to RGB 
    b,g,r,a = cv2.split(img_to_overlay_t)
    overlay_color = cv2.merge((b,g,r))

    # Apply some simple filtering to remove edge noise
    mask = cv2.medianBlur(a,5)

    h, w, _ = overlay_color.shape
    roi = bg_img[y:y+h, x:x+w]

    # Black-out the area behind the logo in our original ROI
    img1_bg = cv2.bitwise_and(roi.copy(),roi.copy(),mask = cv2.bitwise_not(mask))
    
    # Mask out the logo from the logo image.
    img2_bg = cv2.bitwise_and(overlay_color,overlay_color,mask = mask)

    # Update the original image with our new ROI
    bg_img[y:y+h, x:x+w] = cv2.add(img1_bg, img2_bg)

    return bg_img
    
def calibration(detector, predictor):
    """Helper function for determing mean and std"""
    
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (20,50)
    bottomRightCornerOfText = (20,100)
    fontScale              = 1
    fontColor              = (0,0,255)
    lineType               = 2
    data = []
    cap=cv2.VideoCapture(0)
    
    cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'))
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 2880)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1800)
    
    timeout = 300   # [seconds]
    face_flag = 1
    initial_flag = 1
    timeout_start = time.time()
    facemask_flag = True
    beep_flag = True

    while time.time() < timeout_start + timeout:
        sec = int(timeout/10)
        
#         #read facemask and beep flag
#         if keyboard.is_pressed('f'):
#             if facemask_flag:
#                 facemask_flag = False
#             else:
#                 facemask_flag = True
                
#         if keyboard.is_pressed('s'):
#             if beep_flag:
#                 beep_flag = False
#             else:
#                 beep_flag = True
        
        # Getting out image by webcam 
        _, image = cap.read()
#         dim = image.shape
#         print(dim)
#         image = cv2.resize(image, (0,0), fx = 2.25, fy = 2.5, interpolation = cv2.INTER_CUBIC)
        # Converting the image to gray scale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Get faces into webcam's image
        rects = detector(image, 0)
        
        # Show the image
        cv2.imshow("Output", image)
        (x, y, w, h) = cv2.getWindowImageRect('Output')
        
        # Draw white background rectangle
        cv2.rectangle(image, (0, 0), (int(0.28 * w), int(0.18 * h)), (255,255,255), -1)
        cv2.putText(image,"Calibrating...", bottomLeftCornerOfText, font, fontScale, fontColor,lineType)
        cv2.putText(image,"Remaining time: " + str(sec) + "s", bottomRightCornerOfText, font, fontScale, fontColor,lineType)
        
#         # Draw white background rectangle
#         cv2.rectangle(image, (int(0.45 * w), 0), (w, int(0.18 * h)), (255,255,255), -1)
#         if facemask_flag:
#             cv2.putText(image,"Image alert: ON | Press 'F' to turn on/off", (int(0.45 * w) + 10, 50), font, fontScale, fontColor,lineType)
#         else:
#             cv2.putText(image,"Image alert: OFF | Press 'F' to turn on/off", (int(0.45 * w) + 10, 50), font, fontScale, fontColor,lineType)
#         if beep_flag:
#             cv2.putText(image,"Audio alert: ON | Press 'S' to turn on/off", (int(0.45 * w) + 10, 100), font, fontScale, fontColor,lineType)
#         else:
#             cv2.putText(image,"Audio alert: OFF | Press 'S' to turn on/off", (int(0.45 * w) + 10, 100), font, fontScale, fontColor,lineType)
            
        # Draw instruction
        cv2.rectangle(image, (int(0.1 * w), int(0.85 * h)), (int(0.9 * w), h), (255,255,255), -1)
        cv2.putText(image,"Please look at the screen and keep your face at the center", (int(0.1 * w) + 20, int(0.9 * h)), font, fontScale, fontColor,lineType)
        cv2.putText(image,"of the screen for 30 seconds.", (int(0.1 * w) + 20, int(0.9 * h) + 50), font, fontScale, fontColor,lineType)
        
        
        # For each detected face, find the landmark.
        for (i, rect) in enumerate(rects):
            # Make the prediction and transfom it to numpy array
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)
            data.append(shape)
            # Draw on our image, all the finded cordinate points (x,y) 
            for (x, y) in shape:
                cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
        
        if initial_flag == 1:
            initial_flag = 0
            if facemask_flag:
                overlay_t = cv2.imread('head.png',-1) # -1 loads with transparency
                cv2.imshow('Output', overlay_transparent(image, overlay_t, 0, 0, (w,h)))     
        
        # Draw a rectangle around the faces
        if len(rects) == 0 and face_flag != 0:
            cv2.imshow('Output',image)
            face_flag += 1
            if facemask_flag:
                overlay_t = cv2.imread('head.png',-1) # -1 loads with transparency
                cv2.imshow('Output', overlay_transparent(image, overlay_t, 0, 0, (w,h)))
        
        for (i, rect) in enumerate(rects):
            if (rect.left() >= 0.2 * w and rect.right() <= 0.8 * w) and face_flag == 0:
                face_flag = 1
                if facemask_flag:
                    overlay_td = cv2.imread('head_ring.png',-1)
                    cv2.imshow('Output',overlay_transparent(image, overlay_td, 0, 0, (w,h)))
            elif rect.left() >= 0.2 * w and rect.left() <= 0.8 * w:
                cv2.imshow('Output',image)
                face_flag = 1
            else:
                face_flag += 1
                
        if face_flag >= 10:
            face_flag = 1 
            if beep_flag:
                playsound('beep.wav')
            if facemask_flag:
                overlay_t = cv2.imread('head.png',-1) # -1 loads with transparency
                cv2.imshow('Output',overlay_transparent(image, overlay_t, 0, 0, (w,h)))        
        
        timeout = timeout - 1;
        k = cv2.waitKey(5) & 0xFF
        if k == 27:
            break
    
    cv2.destroyAllWindows()
    cap.release()
    cv2.waitKey(1) 
    cv2.waitKey(1) 
    cv2.waitKey(1) 
    cv2.waitKey(1)
    
    features_test = []
    for d in data:
        eye = d[36:68]
        ear = eye_aspect_ratio(eye)
        mar = mouth_aspect_ratio(eye)
        cir = pupil_circularity(eye)
        mouth_eye = mouth_over_eye(eye)
        features_test.append([ear, mar, cir, mouth_eye])
    
    features_test = np.array(features_test)
    x = features_test
    y = pd.DataFrame(x, columns=["EAR","MAR","Circularity","MOE"])
    df_means = y.mean(axis=0)
    df_std = y.std(axis=0)
    
    return df_means, df_std

In [13]:
# Read in dlibs shape predictor for detecting facial landmarks
p = r"shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

In [14]:
# read in feature data
df = pd.read_csv(r'featuresinfo.csv',sep=',')
participants = set(df.Participant)
df = df.drop(["Participant"], axis=1)
df = df[df.Y != 5.0]  # form a binary problem, delete the 5.0 ones
df.loc[df.Y == 0.0, "Y"] = int(0)
df.loc[df.Y == 10.0, "Y"] = int(1)

train_percentage = 14/18 # based on max uniq. participants
train_samples = int(len(df) * train_percentage)
test_samples = len(df) - train_samples

df_train = df[:train_samples]
df_test = df[-test_samples:]

X_test = df_test.drop(["Y"], axis=1)
y_test = df_test["Y"]

X_train = df_train.drop('Y', axis=1)
y_train = df_train['Y']
print(f'X_test: {X_test.shape} \ny_test: {y_test.shape} \nX_train: {X_train.shape} \ny_train: {y_train.shape}')
df_train

X_test: (1920, 8) 
y_test: (1920,) 
X_train: (6720, 8) 
y_train: (6720,)


,Y,MOE,EAR,MAR,Circularity,EAR_N,MAR_N,Circularity_N,MOE_N
0,0.0,2.534145,0.312688,0.792397,0.432896,-0.999551,0.802437,-0.948848,0.877172
1,0.0,2.379947,0.321940,0.766199,0.486923,-0.000897,0.317858,1.044306,0.211742
2,0.0,2.078550,0.331216,0.688449,0.456029,1.000448,-1.120295,-0.095458,-1.088914
3,0.0,2.509767,0.309246,0.776136,0.452655,-1.371052,0.501666,-0.219920,0.771970
4,0.0,3.586821,0.204691,0.734189,0.361123,-12.657205,-0.274242,-3.596686,5.419921
...,...,...,...,...,...,...,...,...,...
9595,1.0,3.516036,0.308254,1.083833,0.440628,-3.087456,3.700493,-1.776452,4.040717
9596,1.0,3.585545,0.301381,1.080614,0.447949,-3.694883,3.646635,-1.472575,4.322363
9597,1.0,3.750118,0.286855,1.075740,0.450278,-4.978551,3.565087,-1.375905,4.989206
9598,1.0,4.123695,0.260188,1.072936,0.403545,-7.335156,3.518178,-3.315660,6.502924


In [15]:
#KNN classifier
def average(y_pred):
    """Averaging sequential frames for classifier"""
    for i in range(len(y_pred)):
        if i % 240 == 0 or (i+1) % 240 == 0:
            pass
        else: 
            average = float(y_pred[i-1] + y_pred[i] + y_pred[i+1])/3
            if average >= 0.5:
                y_pred[i] = 1
            else:
                y_pred[i] = 0
    return y_pred

acc3_list = []
f1_score3_list = []
roc_3_list = []

# take 45 runs and save best one
for i in range(1, 45):
    neigh = KNeighborsClassifier(n_neighbors=i)
    neigh.fit(X_train, y_train) 
    pred_KN = neigh.predict(X_test)
    pred_KN = average(pred_KN)
    y_score_3 = neigh.predict_proba(X_test)[:,1]
    acc3_list.append(accuracy_score(y_test, pred_KN))
    f1_score3_list.append(metrics.f1_score(y_test, pred_KN))
    roc_3_list.append(metrics.roc_auc_score(y_test, y_score_3))
    
neigh = KNeighborsClassifier(n_neighbors=acc3_list.index(max(acc3_list))+1)
print(f"Neighbors: {neigh.get_params()['n_neighbors']}")
neigh.fit(X_train, y_train) 

Neighbors: 38


KNeighborsClassifier(n_neighbors=38)

In [16]:
def model_knn(landmarks):
    """Returns features and classification result"""
    features = pd.DataFrame(columns=["EAR","MAR","Circularity","MOE"])
    eye = landmarks[36:68] # Extracting relevant parts (eyes + mouth)
    ear = eye_aspect_ratio(eye)
    mar = mouth_aspect_ratio(eye)
    cir = pupil_circularity(eye)
    mouth_eye = mouth_over_eye(eye)
    df = features.append({"EAR":ear,"MAR": mar,"Circularity": cir,"MOE": mouth_eye},ignore_index=True)

    # Normalisation
    df["EAR_N"] = (df["EAR"] - mean["EAR"]) / std["EAR"]
    df["MAR_N"] = (df["MAR"] - mean["MAR"]) / std["MAR"]
    df["Circularity_N"] = (df["Circularity"] - mean["Circularity"]) / std["Circularity"]
    df["MOE_N"] = (df["MOE"] - mean["MOE"]) / std["MOE"]
    
    Result = neigh.predict(df)  
    UDP_IP = "172.20.10.2"
    UDP_PORT = 5005
    sock = socket.socket(socket.AF_INET, # Internet
                        socket.SOCK_DGRAM) # UDP
    
    if Pre_State == 2 or Pre_State != Result:
        Pre_State = Result
        if Result == 1:
            MESSAGE = b"Alertness level: 0"
        else:
            MESSAGE = b"Alertness level: 1"
    sock.sendto(MESSAGE, (UDP_IP, UDP_PORT))        
    
    if Result == 1:
        Result_String = "Drowsy"
        data = bytes("1", "utf-8")         # Unicode data
        fontColor = (0, 0, 245)
    else:
        Result_String = "Alert"
        data = bytes("0", "utf-8")         # Unicode data
        fontColor = (255, 255, 255)
    sock.sendto(data, (UDP_IP, UDP_PORT))
    return Result_String, df.values, fontColor

In [17]:
def live():
    """real-time alertness recognition"""
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (20, 50)
    fontScale              = 1
    lineType               = 2
    data = []
    result = []
    face_flag = 0
    initial_flag = 1
    cap=cv2.VideoCapture(0)
    Pre_State = 2
    while True:
        # Getting out image by webcam 
        _, image = cap.read()
        # Converting the image to gray scale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Get faces into webcam's image
        rects = detector(image, 0)

        # For each detected face, find the landmark.
        for (i, rect) in enumerate(rects):
            # Make the prediction and transfom it to numpy array
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)
#             Result_String, features, fontColor = model_knn(shape) 
            
            features = pd.DataFrame(columns=["EAR","MAR","Circularity","MOE"])
            eye = shape[36:68] # Extracting relevant parts (eyes + mouth)
            ear = eye_aspect_ratio(eye)
            mar = mouth_aspect_ratio(eye)
            cir = pupil_circularity(eye)
            mouth_eye = mouth_over_eye(eye)
            df = features.append({"EAR":ear,"MAR": mar,"Circularity": cir,"MOE": mouth_eye},ignore_index=True)

            # Normalisation
            df["EAR_N"] = (df["EAR"] - mean["EAR"]) / std["EAR"]
            df["MAR_N"] = (df["MAR"] - mean["MAR"]) / std["MAR"]
            df["Circularity_N"] = (df["Circularity"] - mean["Circularity"]) / std["Circularity"]
            df["MOE_N"] = (df["MOE"] - mean["MOE"]) / std["MOE"]
    
            Result = neigh.predict(df)  
            UDP_IP = "127.0.0.1"
            UDP_PORT = 6998
            sock = socket.socket(socket.AF_INET, # Internet
                        socket.SOCK_DGRAM) # UDP
            
            if Result == 1:
                Result_String = "Drowsy"
                data = bytes("1", "utf-8")         # Unicode data
                fontColor = (0, 0, 245)
            else:
                Result_String = "Alert"
                data = bytes("0", "utf-8")         # Unicode data
                fontColor = (255, 255, 255)
            sock.sendto(data, (UDP_IP, UDP_PORT))
    
#             if Pre_State == 2 or Pre_State != Result:
#                 Pre_State = Result
#                 if Result == 1:
#                     Result_String = "Drowsy"
#                     data = bytes("1", "utf-8")         # Unicode data
#                     fontColor = (0, 0, 245)
#                 else:
#                     Result_String = "Alert"
#                     data = bytes("0", "utf-8")         # Unicode data
#                     fontColor = (255, 255, 255)
#                 sock.sendto(data, (UDP_IP, UDP_PORT))        
            
            cv2.putText(image, Result_String, bottomLeftCornerOfText, font, fontScale, fontColor,lineType)
            #data.append (features)
            #result.append(Result_String)
    
            # Draw on our image, all the finded cordinate points (x,y) 
            for (x, y) in shape:
                cv2.circle(image, (x, y), 2, (0, 255, 0), -1) # maybe changing red if drowsy, otherwise green
                
        # Show the image
        cv2.imshow("Output", image)
        (x, y, w, h) = cv2.getWindowImageRect('Output')
        
        if initial_flag == 1:
            overlay_t = cv2.imread('head.png',-1) # -1 loads with transparency
            cv2.imshow('Output', overlay_transparent(image, overlay_t, 0, 0, (w,h)))
            initial_flag = 0
        
        # Draw a rectangle around the faces
        if len(rects) == 0:
            cv2.imshow("Output", image)
            overlay_t = cv2.imread('head.png',-1) # -1 loads with transparency
            cv2.imshow('Output', overlay_transparent(image, overlay_t, 0, 0, (w,h)))
            face_flag += 1
        
        for (i, rect) in enumerate(rects):
            if (rect.left() >= 0.2 * w and rect.left() <= 0.8 * w) and face_flag == 0:
                overlay_td = cv2.imread('head_ring.png',-1)
                cv2.imshow('Output',overlay_transparent(image, overlay_td, 0, 0, (w,h)))
                face_flag = 1
            elif rect.left() >= 0.2 * w and rect.left() <= 0.8 * w:
                cv2.imshow('Output',image)
                face_flag = 1
            else:
                face_flag += 1
                
        if face_flag >= 10:
            playsound('beep.wav')
            overlay_t = cv2.imread('head.png',-1) # -1 loads with transparency
            cv2.imshow('Output',overlay_transparent(image, overlay_t, 0, 0, (w,h)))
            face_flag = 1
        
        k = cv2.waitKey(5) & 0xFF
        if k == 27:
            break

    cv2.destroyAllWindows()
    cap.release()
    cv2.waitKey(1) 
    cv2.waitKey(1) 
    cv2.waitKey(1) 
    cv2.waitKey(1)
    
    return data, result

In [28]:
mean, std = calibration(detector, predictor)

In [ ]:
#Run Demonstration with KNN classifier
features, result = live()